In [1]:
## QICK LIBRARY
from qick import *

## tProc Assembler
from qick.tprocv2_assembler import Assembler, Logger

import qick.tprocv2_assembler as qasm
from importlib import reload  # Python 3.4+

import numpy as np
import time

##### Load FGPA BitStream
soc = QickSoc('./qick_4x2_jac.bit')

In [2]:
reload(qasm)

asm = """
//TEST program

REG_WR w_freq   imm #60000000
REG_WR w_phase  imm  #0
REG_WR w_env    imm  #0
REG_WR w_gain   imm #20000
REG_WR w_length imm #200

//REG_WR w_conf   imm  #1  // NoPHRST - NoPeriodic - DDS
//REG_WR w_conf   imm  #5  // NoPHRST - Periodic   - DDS
REG_WR w_conf   imm  #17 // PHRST   - NoPeriodic - DDS
//REG_WR w_conf   imm  #21 // PHRST   - Periodic   - DDS

TIME inc_ref #15000


WPORT_WR p0 r_wave
WPORT_WR p1 r_wave


TRIG p0 set
TRIG p1 set
TRIG p0 clr
TRIG p0 set
TRIG p1 set
TRIG p2 set
TRIG p3 set


TRIG p0 clr
TRIG p1 clr
TRIG p2 clr
TRIG p3 clr

END: 
    JUMP END

"""

p_txt, p_bin             = Assembler.str_asm2bin(asm)
soc.tproc.Load_PMEM(p_bin)

#The ZCU216 has a tproc_start and tproc_stop
soc.tproc.start()
#soc.tproc.axi()


In [ ]:
from pynq.buffer import allocate
addr = 0
length = 2

buff_rd = allocate(shape=(length, 1), dtype=np.int32)

print('LOAD VALUES 1')
soc.qtt.mem_addr        = addr
soc.qtt.mem_len         = length


print('ANTES')
soc.qtt.print_debug()
soc.qtt.print_debug()


print('DMA read 0')
#Start operation
soc.qtt.qtt_ctrl         = 4
print('DMA read 1')
soc.qtt.dma.recvchannel.transfer(buff_rd, nbytes=int(length*4))
print('DMA read 2')
#soc.qtt.dma.recvchannel.wait()
print('DMA read 3')
soc.qtt.print_debug()
soc.qtt.print_debug()


In [ ]:
print(buff_rd)
print(len(buff_rd))

In [ ]:
# WHen READ check READ SIZE IS SMALL THAN DMA_QTY...

soc.qtt.print_axi_regs()
soc.qtt.print_debug()


In [ ]:
soc.qtt.print_debug()
soc.qtt.print_debug()


In [ ]:
#soc.qtt.info()